In [107]:
import scipy.io
import re
import numpy as np
from collections import Counter
import os
import pandas as pd
import mne

In [108]:
dir = './eeg_feature_smooth/1/2_20150915.mat'
mat = scipy.io.loadmat(dir)

mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Nov 15 18:20:31 2018',
 '__version__': '1.0',
 '__globals__': [],
 'de_movingAve1': array([[[24.32216922, 23.8299595 , 21.99265309, 20.36229144,
          19.28716845],
         [26.3810962 , 23.27124132, 21.83348461, 20.40642429,
          18.99897358],
         [26.1975243 , 24.19996808, 22.14043615, 20.21318533,
          19.25527881],
         ...,
         [27.94988434, 25.05097226, 21.92035846, 19.73154362,
          19.01266835],
         [28.01661998, 24.94609678, 21.80387392, 19.67515291,
          18.8925514 ],
         [28.11727868, 24.98763241, 21.7766299 , 19.73756256,
          19.04933103]],
 
        [[25.33822895, 24.28766666, 22.60570288, 19.84935264,
          18.91796159],
         [26.02279509, 23.47917347, 21.6432694 , 19.89491882,
          19.09822529],
         [25.76399118, 24.12428874, 21.76357899, 20.26235175,
          18.96567316],
         ...,
         [28.22600252, 25.21472747, 22.

In [84]:
dato = mat['ha_eeg1']
print(dato.shape)
dato

(62, 33601)


array([[ -4.67896461, -22.47095108,  -7.24196434, ...,  26.91149712,
         21.45767212,  26.7624855 ],
       [  3.66568565, -15.22898674,  -1.87754631, ...,   8.97049904,
          1.49011612,   9.4473362 ],
       [  9.50694084,  -4.85777855,   3.27825546, ...,  12.66598701,
          9.77516174,  15.7058239 ],
       ...,
       [ -0.68545341,  -1.43051147,  -2.62260437, ...,  -0.95367432,
          0.29802322,  -0.35762787],
       [ -3.15904617,  -9.59634781,  -5.78165054, ...,   3.42726707,
          5.03659248,   4.17232513],
       [ -2.86102295,  -9.95397568,  -5.7220459 , ...,   4.17232513,
          5.12599945,   3.51667404]])

In [111]:
train_label = [1,2,3,0,2,0,0,1,0,1,2,1,2,3,3,3,     1,2,3,0,2,0,0,1,0,1,2,1,2,3,3,3]
valid_label = [1,2,0,3,    1,2,0,3]
test_label = [1,2,0,3,    1,2,0,3]
train = ['tyc_eeg1', 'tyc_eeg2', 'tyc_eeg3', 'tyc_eeg4', 'tyc_eeg5', 'tyc_eeg6', 'tyc_eeg7', 'tyc_eeg8', 'tyc_eeg9', 'tyc_eeg10', 'tyc_eeg11', 'tyc_eeg12', 'tyc_eeg15', 'tyc_eeg16', 'tyc_eeg19', 'tyc_eeg20',
        'ha_eeg1', 'ha_eeg2', 'ha_eeg3', 'ha_eeg4', 'ha_eeg5', 'ha_eeg6', 'ha_eeg7', 'ha_eeg8', 'ha_eeg9', 'ha_eeg10', 'ha_eeg11', 'ha_eeg12', 'ha_eeg15', 'ha_eeg16', 'ha_eeg19', 'ha_eeg20']
valid = ['tyc_eeg13', 'tyc_eeg17', 'tyc_eeg21', 'tyc_eeg22',
        'ha_eeg13', 'ha_eeg17', 'ha_eeg21', 'ha_eeg22']
test = ['tyc_eeg14', 'tyc_eeg18', 'tyc_eeg23', 'tyc_eeg24',
       'ha_eeg14', 'ha_eeg18', 'ha_eeg23', 'ha_eeg24']

In [89]:
def limpiar_ruido(X):
    # Hacemos un procesado ICA
    sampling_rate = 1000
    info = mne.create_info(ch_names=[f"ch_eeg{i + 1}" for i in range(X.shape[0])],
                        sfreq=sampling_rate,
                        ch_types='eeg')
    raw = mne.io.RawArray(X, info)
    raw.resample(200, npad="auto")
    raw = raw.filter(1, 30)
    raw.set_eeg_reference('average', projection=True).apply_proj() 

    ica = mne.preprocessing.ICA(n_components=20, random_state=42)
    ica.fit(raw, decim=3)
    ica.apply(raw)

    return raw.get_data().T

In [112]:
#dir = './eeg_raw_data/1/10_20151014.mat'
#direc = ['./eeg_raw_data/1/10_20151014.mat', './eeg_raw_data/1/2_20150915.mat']
#direc = ['./eeg_raw_data/1/10_20151014.mat']
#mat = scipy.io.loadmat(dir)
direc = ['./eeg_feature_smooth/1/10_20151014.mat', './eeg_feature_smooth/1/2_20150915.mat']

X_train = np.array([])
X_valid = np.array([])
X_test = np.array([])

y_train = []
y_valid = []
y_test = []

c1 = 0
c2 = 0
c3 = 0

for d in direc:
    mat = scipy.io.loadmat(d)
    for arch,lab in zip(train, train_label):
        if arch in mat:
            dato = mat[arch]
            dato = limpiar_ruido(dato)
            if c1 == 0:
                X_train = dato
                c1 += 1
            else:
                X_train = np.concatenate((X_train, dato), axis=0)
            y_train.extend([lab] * dato.shape[0])
    for arch,lab in zip(valid, valid_label):
        if arch in mat:
            dato = mat[arch]
            dato = limpiar_ruido(dato)
            if c2 == 0:
                X_valid = dato
                c2 += 1
            else:
                X_valid = np.concatenate((X_valid, dato), axis=0)
            y_valid.extend([lab] * dato.shape[0])
    for arch,lab in zip(test, test_label):
        if arch in mat:
            dato = mat[arch]
            dato = limpiar_ruido(dato)
            if c3 == 0:
                X_test = dato
                c3 += 1
            else:
                X_test = np.concatenate((X_test, dato), axis=0)
            y_test.extend([lab] * dato.shape[0])

In [113]:
print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)
print(len(y_train))
print(len(y_valid))
print(len(y_test))

(0,)
(0,)
(0,)
0
0
0


In [101]:
print(len(y_train))
print(len(y_valid))
print(len(y_test))

conteo = {}
for elemento in y_train:
    conteo[elemento] = conteo.get(elemento, 0) + 1
print(conteo)

181280
40720
52800
{1: 52480, 2: 39360, 3: 42240, 0: 47200}


In [102]:
def apply_sliding_window(X, y, window_size, step_size):
    y_res = []
    X_res = []
    posicion = 0
    stride = window_size - step_size
    total = len(y)
    
    while (posicion + window_size < total):
        y_aux = []
        X_aux = []
        
        for _ in range(window_size):
            y_aux.append(y[posicion])
            X_aux.append(X[posicion])
            posicion += 1
        
        valor_mas_comun = Counter(y_aux).most_common(1)
        y_res.append(valor_mas_comun[0][0])

        X_res.append(np.asarray(X_aux))

        posicion -= stride    
    
    return np.asarray(X_res), np.asarray(y_res)

In [103]:
X_train, y_train = apply_sliding_window(X_train, y_train, window_size=64, step_size=8)
X_valid, y_valid = apply_sliding_window(X_valid, y_valid, window_size=64, step_size=8)
X_test, y_test = apply_sliding_window(X_test, y_test, window_size=64, step_size=8)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

(22652, 64, 62)
(5082, 64, 62)
(6592, 64, 62)


In [104]:
print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(22652,)
(5082,)
(6592,)


In [105]:
lista_final = [X_train, X_valid, X_test, y_train, y_valid, y_test]

with open('dataset_X_y_sliding_window.npy', 'wb') as f:
    np.save(f, np.array(lista_final, dtype=object))